## AI1103: Programming with AI

### Project: House price prediction dataset

Source: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

Author: Tanmay Goyal, AI20BTECH11021

#### 1. Obtaining the prices manually

In [1]:
# importing the required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# the following would be used to cross-check our results
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [2]:
# we would first read the train.csv file into train_df
train_df = pd.read_csv("train.csv")

# printing the shape of train_df for our reference
print(train_df.shape)

# printing the first few rows of train_df for our reference
train_df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# reading the file test.csv into test_df
test_df = pd.read_csv("test.csv")

# printing the shape of test_df for our reference
print(test_df.shape)

# printing the first few rows of test_df for our reference
test_df.head()

(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
# A lot of the columns have Nans, so our first issue would be dealing with these

# we first obtain a list of all columns having more than 50 Nans. Because of the larger number of Nans, we can safely
# assume these columns would not contribute much towards the Price of the houses
columns_with_Nans = train_df.isna().sum()>50

# we now drop those columns from train_df
for column in train_df.columns:
    if columns_with_Nans[column] == True:
        train_df.drop(column, axis=1 , inplace=True)

# Those entries that still have Nans are now filled with the most common value from that specific column, since 
# the most common entry would be exerting larger influence 
train_df = train_df.apply(lambda x : x.fillna(x.value_counts().index[0]))

# we check if any Nans remain in train_df
print(train_df.isna().any().sum())

# Finally, we drop the ID column, since it is only indexing our houses
train_df.drop(['Id'] , axis = 1, inplace = True)

0


In [5]:
# we would now remove those columns from test_df as well, which we assumed will not be 
# contributing significantly to the prices of the houses
for column in test_df.columns:
    if columns_with_Nans[column] == True:
        test_df.drop(column, axis=1 , inplace=True)

# again, we fill in the remaining Nans with the most common values from their columns
test_df = test_df.apply(lambda x : x.fillna(x.value_counts().index[0]))

# we check for any remaining Nans in test_df
print(test_df.isna().any().sum())

# we save the ID column from test_df for future references
test_id = test_df['Id']

# we drop the ID column from test_df
test_df.drop(['Id'] , axis=1 , inplace = True)

0


In [6]:
# the next issue we deal with is having categorical variables for regression.

#we convert these variables into dummy variables for both train_df and test_df
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)


In [7]:
# we then try to find the 20 most important features affecting the sale price of a house

# we first find the correlation of every column with the SalePrice
corr_list = np.array([train_df['SalePrice'].corr(train_df[column]) for column in train_df.columns])

# we then sort these correlations, and make a list of the indices of all columns in order of the sorted list
indices = np.argsort(corr_list)

# we extract the 20 highest values
# we exclude index -1 because that would be SalePrice column having correlation 1 with itself
max_corr = indices[-21:-1]


# we then extract the columns having highest correlation with SalePrice
feature_columns = [train_df.columns[i] for i in max_corr]

# we save the SalePrice column seperately for further use
sale_price = train_df['SalePrice']

# we remove the SalePrice column from train_df
train_df.drop(['SalePrice'] , axis = 1 , inplace = True)

In [8]:
# we now check if there are any extra columns in train_df
print(set(train_df.columns) - set(test_df.columns))

{'Exterior2nd_Other', 'Condition2_RRAe', 'HouseStyle_2.5Fin', 'Exterior1st_Stone', 'Condition2_RRNn', 'Heating_Floor', 'RoofMatl_Membran', 'Condition2_RRAn', 'Heating_OthW', 'RoofMatl_Roll', 'RoofMatl_ClyTile', 'Exterior1st_ImStucc', 'Electrical_Mix', 'RoofMatl_Metal', 'Utilities_NoSeWa'}


In [9]:
# the extra columns in train_df are now added to test_df, since we want both to have the same columns
for idx , column in enumerate(train_df.columns):
    if column not in test_df.columns:
        test_df.insert(loc = idx , column = column , value = 0) 
        #since we know none of the rows hold that label of categorical variable
    
# we once again check if there remain any columns in train_df which are not found in test_df
print(set(train_df.columns) - set(test_df.columns))

set()


In [10]:
# finally, if there are any extra columns in test_df, which are not in train_df, they are dropped
# since, our primary focus is train_df
for idx , column in enumerate(test_df.columns):
    if column not in train_df.columns:
       test_df.drop(column, axis=1, inplace=True)

In [11]:
#We make sure columns in test_df are in same order as that in train_df
test_df = test_df[train_df.columns]

#We check if the order is same
print(train_df.columns == test_df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [12]:
# we convert our sale_price to np array for calculations
y_train = np.array(sale_price)

# we extract the 20 columns having maximum impact on saleprice as our X_train
X_train = np.array(train_df[feature_columns])

# we extract the same 20 columns having maximum impact on saleprice as our X_test
X_test = np.array(test_df[feature_columns])

# we add a column of ones to accomodate the bias term for both X_train and X_test
X_train = np.hstack((np.ones((X_train.shape[0] , 1)), X_train))
X_test = np.hstack((np.ones((X_test.shape[0] , 1)), X_test))

For a linear regression, we know the optimal set of weights is given by:

$\beta ^ * = (X^T  X) ^{-1}  X^T  y$

where $(X^T  X) ^{-1}$ is called as the pseudo-inverse of $X$.

In [13]:
#calculating the psuedo- inverse of X
pseudo_inv = np.linalg.inv(np.matmul(X_train.T , X_train))

#calculating the optimal set of weights using the equation mentioned above
beta = pseudo_inv @ X_train.T @ y_train

#checking if number of columns of beta match number of columns in X
print(beta.shape[0] , X_train.shape[1])


21 21


In [14]:
# finally, we save our predictions under result_m
result_m = X_test @ beta
result_m

array([102787.06162586, 139660.42990578, 179900.90111694, ...,
       154926.48445969, 113714.73141107, 222990.68043667])

#### Obtaining the solution using in- built libraries

In [15]:
# we first extract the train_df and test_df values 
x_train = train_df.values
x_test = test_df.values

In [16]:
# we transform all data into zero mean and unit variance
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [17]:
# since we had used 20 components for obtaining the solution manually, 
# we would decompose the data into 20 components as well
pca = PCA(n_components = 20)

In [18]:
# we fit the training data to this pca model
pca.fit(x_train)

# we now apply the transform to both testing and training data
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


In [19]:
# setting up the Linear Regression model
model=LinearRegression()

# we fit the training data, as well as the target variable SalePrice to this model
model.fit(x_train,sale_price)

LinearRegression()

In [20]:
# we now predict the outputs for the testing data and store it in result_scikit
result_scikit = model.predict(x_test)
result_scikit

array([104745.69147659, 178405.89753071, 186087.28250348, ...,
       165406.92050327, 114453.83201366, 229369.80124488])

In [21]:
# we compile both the results to compare

final_result = pd.DataFrame({'Id' : test_id , 'Results computed Manually' : result_m , 'Results using Scikit Learn' : result_scikit})
final_result

,Id,Results computed Manually,Results using Scikit Learn
0,1461,102787.061626,104745.691477
1,1462,139660.429906,178405.897531
2,1463,179900.901117,186087.282503
3,1464,194346.068237,212029.115088
4,1465,191294.322196,188832.328053
...,...,...,...
1454,2915,81511.108838,89444.483876
1455,2916,93768.489789,99935.326326
1456,2917,154926.484460,165406.920503
1457,2918,113714.731411,114453.832014


In [22]:
# We also calculate the RMSE error
error = np.sum((np.array(result_scikit)-np.array(result_m))**2)
error/= final_result.shape[0]
error = error ** 0.5
print("RMSE error: {:.2f}".format(error))

RMSE error: 18612.39


### RMSE vs Number of components:

As the number of components we consider increases, the accuracy of the model increases, and the RMSE decreases 

In [23]:
# copying the relevant snippets of code from above    
components = [i for i in range (1,51)]
errors = []

for n in range(1,51):    
    error = 0
    train_df = pd.read_csv("train.csv")
    test_df = pd.read_csv("test.csv")
    
    columns_with_Nans = train_df.isna().sum()>50

    for column in train_df.columns:
        if columns_with_Nans[column] == True:
            train_df.drop(column, axis=1 , inplace=True)

    train_df = train_df.apply(lambda x : x.fillna(x.value_counts().index[0]))
    train_df.drop(['Id'] , axis = 1, inplace = True)

    for column in test_df.columns:
        if columns_with_Nans[column] == True:
            test_df.drop(column, axis=1 , inplace=True)

    test_df = test_df.apply(lambda x : x.fillna(x.value_counts().index[0]))
    test_df.drop(['Id'] , axis=1 , inplace = True)
    
    train_df = pd.get_dummies(train_df)
    test_df = pd.get_dummies(test_df)

    corr_list = np.array([train_df['SalePrice'].corr(train_df[column]) for column in train_df.columns])
    indices = np.argsort(corr_list)
    max_corr = indices[(-1-n):-1]
    feature_columns = [train_df.columns[i] for i in max_corr]

    sale_price = train_df['SalePrice']
    train_df.drop(['SalePrice'] , axis = 1 , inplace = True)

    for idx , column in enumerate(train_df.columns):
        if column not in test_df.columns:
            test_df.insert(loc = idx , column = column , value = 0) 
        
   
    for idx , column in enumerate(test_df.columns):
        if column not in train_df.columns:
            test_df.drop(column, axis=1, inplace=True)
    
    test_df = test_df[train_df.columns]

    y_train = np.array(sale_price)
    X_train = np.array(train_df[feature_columns])
    X_test = np.array(test_df[feature_columns])
    X_train = np.hstack((np.ones((X_train.shape[0] , 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0] , 1)), X_test))

    pseudo_inv = np.linalg.inv(np.matmul(X_train.T , X_train))
    beta = pseudo_inv @ X_train.T @ y_train

    result_m = X_test @ beta
    

    x_train = train_df.values
    x_test = test_df.values
    x_train = StandardScaler().fit_transform(x_train)
    x_test = StandardScaler().fit_transform(x_test)
    
    pca = PCA(n_components = n)
    pca.fit(x_train)

    x_train = pca.transform(x_train)
    x_test = pca.transform(x_test)

    model=LinearRegression()
    model.fit(x_train,sale_price)
    result_scikit = model.predict(x_test)
   

    error = np.sum((np.array(result_scikit)-np.array(result_m))**2)
    error/= final_result.shape[0]
    error = error ** 0.5
    errors.append(error)

plt.plot(components , errors , 'r' , label = "RSME")
plt.grid(True)
plt.legend()
plt.xlabel("Number of Principal Components")
plt.ylabel("RSME")
plt.title("RSME vs Number of Principal Components")
plt.show()    